In [1]:
%%capture
###########ПОДГОТОВКА################
p = 'п.2' ##версия таблиц
deli = 11 ##количество цветов фильтра
#####################################

##Загрузка библиотек##
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from shapely.geometry import Polygon
from folium.plugins import HeatMap
import ipywidgets as widgets
!pip install osmnx --user
import geopandas as gpd
from shapely import wkt
import pandas as pd
import numpy as np
import osmnx as ox
import warnings
import folium
import random
import math
import json
import h3

warnings.filterwarnings("ignore")

##Считывание файла##
hexo3 = pd.read_excel('C://Users//EliseevAA//Desktop//Проекты//SBER//Hex//2ГИС_' + p + ' т.3.xlsx')
hexo4 = pd.read_excel('C://Users//EliseevAA//Desktop//Проекты//SBER//Hex//2ГИС_' + p + ' т.4.xlsx')
hexo5 = pd.read_excel('C://Users//EliseevAA//Desktop//Проекты//SBER//Hex//2ГИС_' + p + ' т.5.xlsx')
hexo5['REGION_ID'] = hexo3['REGION_ID'][0]
hexo5['ANALYTIC_DATE'] = hexo3['ANALYTIC_DATE'][0]

##Объединение таблицы №3 и №5##
hexo_3_5_cell = pd.concat([hexo3[['CELL_ID', 'REGION_ID', 'INDUSTRY_ID', 'INDUSTRY_NAME', 'ANALYTIC_DATE', 'area']], 
                           hexo5[['CELL_ID', 'REGION_ID', 'INDUSTRY_ID', 'INDUSTRY_NAME', 'ANALYTIC_DATE', 'area']]], 
                           axis = 0).drop_duplicates()

hexo3_merged = hexo_3_5_cell.merge(hexo3, how = 'left', 
                                   on = ['CELL_ID', 'REGION_ID', 'INDUSTRY_ID', 'INDUSTRY_NAME', 'ANALYTIC_DATE', 'area'])
hexo3_5_merged = hexo3_merged.merge(hexo5, how = 'left', 
                                   on = ['CELL_ID', 'REGION_ID', 'INDUSTRY_ID', 'INDUSTRY_NAME', 'ANALYTIC_DATE', 'area'])

##Задание цветовой шкалы для таблицы №3 и № 5##
hexo3_5_merged['size'] = hexo3_5_merged['CELL_ID'].str.count('f')
size_arr = sorted(list(hexo3_5_merged['size'].unique()))
hexo3_5_merged['size'] = hexo3_5_merged['size'].replace(size_arr[0], 'Small').replace(size_arr[1], 'Medium').replace(size_arr[2], 'Big')
coli = [x for x in hexo3_5_merged.columns.to_list() if ('color_id' not in x) 
                                                    and ('COLOR_ID' not in x)]
hexo3_5_merged = hexo3_5_merged[coli] 
coli = coli[6::]
coli.remove('FACTOR_ID')
coli.remove('size') 

hexo35_color = pd.DataFrame()
for siz in list(hexo3_5_merged['size'].unique()):
    for cat in list(hexo3_5_merged['INDUSTRY_NAME'].unique()):
        hexo35 = hexo3_5_merged[(hexo3_5_merged['INDUSTRY_NAME'] == cat) &
                                (hexo3_5_merged['size'] == siz)].copy()
        for id_color in coli:
            ind_min = hexo35[id_color].min()
            ind_max = hexo35[id_color].max()
            ind_median = np.median(hexo35[hexo35[id_color].isna() == False][id_color].unique())
            ind_diff_min = (ind_median - ind_min)/(math.floor(deli/2))
            ind_diff_max = (ind_max - ind_median)/(math.floor(deli/2))

            bins = [ind_min]
            for i in range(math.floor(deli/2)):
                i += 1
                bins.append(ind_min + ind_diff_min*i)
            for i in range(math.floor(deli/2)):
                i += 1
                bins.append(ind_median + ind_diff_max*i)

            labels = list(np.arange(deli))
            new_ind = 'COLOR_' + id_color
            hexo35[new_ind] = np.searchsorted(bins, hexo35[id_color].values).astype('str')
        hexo35_color = pd.concat([hexo35_color, hexo35], axis = 0)

##Задание цветовой шкалы для таблицы №4##
hexo4['size'] = hexo4['CELL_ID'].str.count('f')
size_arr = sorted(list(hexo4['size'].unique()))
hexo4['size'] = hexo4['size'].replace(size_arr[0], 'Small').replace(size_arr[1], 'Medium').replace(size_arr[2], 'Big')

coli = [x for x in hexo4.columns.to_list() if ('color_id' not in x) 
                                           and ('COLOR_ID' not in x)]
hexo4 = hexo4[coli]
hexo4['SPENDING_BIN'] = hexo4['SPENDING_BIN'].astype('str')
coli = coli[5::]
coli.remove('ANALYTIC_DATE')
coli.remove('area')
coli.remove('size') 

hexo4_color = pd.DataFrame()
for siz in list(hexo4['size'].unique()):
    for cat in list(hexo4['INDUSTRY_NAME'].unique()):
        for spend in list(hexo4['SPENDING_BIN'].unique()):
            hexo4_target = hexo4[(hexo4['SPENDING_BIN'] == spend) &
                                 (hexo4['INDUSTRY_NAME'] == cat) &
                                 (hexo4['size'] == siz)].copy()
            for id_color in coli:
                ind_min = hexo4_target[id_color].min()
                ind_max = hexo4_target[id_color].max()
                ind_median = np.median(hexo4_target[hexo4_target[id_color].isna() == False][id_color].unique())
                ind_diff_min = (ind_median - ind_min)/(math.floor(deli/2))
                ind_diff_max = (ind_max - ind_median)/(math.floor(deli/2))

                bins = [ind_min]
                for i in range(math.floor(deli/2)):
                    i += 1
                    bins.append(ind_min + ind_diff_min*i)
                for i in range(math.floor(deli/2)):
                    i += 1
                    bins.append(ind_median + ind_diff_max*i)

                labels = list(np.arange(deli))
                new_ind = 'COLOR_' + id_color
                hexo4_target[new_ind] = np.searchsorted(bins, hexo4_target[id_color].values).astype('str')
            hexo4_color = pd.concat([hexo4_color, hexo4_target], axis = 0) 

##Общее форматирование объединённой таблицы##
hexo = pd.concat([hexo35_color, hexo4_color], axis = 0)

hexo['ANALYTIC_DATE'] = pd.to_datetime(hexo['ANALYTIC_DATE']).astype('str')
hexo['SPENDING_BIN'] = hexo['SPENDING_BIN'].replace(np.nan, 'None')
hexo = hexo.replace(np.nan, '-')

##Формирование функций##
def date_choice(DATE):
    global date_name
    date_name = DATE

def region_choice(REGION):
    global region_name
    region_name = REGION
    
def industry_choice(INDUSTRY):
    global industry_name
    industry_name = INDUSTRY
    
def district_choice(DISTRICT):
    global district_name
    district_name = DISTRICT
    
def category_choice(CATEGORY):
    global category_name
    category_name = CATEGORY

def bin_choice(BIN):
    global bin_name
    bin_name = BIN
    
def size_choice(SIZE):
    global size_name
    size_name = SIZE
    
def visualize_polygons(geometry):
    
    lats, lons = get_lat_lon(geometry)
    
    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')
    
    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)
    return m  

def get_lat_lon(geometry):
        
    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon

#Формирование цветовой и размерных шкал:
dict_color = {
              '0': 'darkred',
              '1': 'firebrick',
              '2': 'red',
              '3': 'indianred',
              '4': 'lightcoral', 
              '5': 'rosybrown',
              '6': 'khaki',
              '7': 'darkkhaki',
              '8': 'yellowgreen',
              '9': 'seagreen',
              '10':'darkgreen'
             }
dict_size = {
              '0': 1,
              '1': 2,
              '2': 3,
              '3': 4,
              '4': 5, 
              '5': 6,
              '6': 7,
              '7': 8,
              '8': 9,
              '9': 10,
              '10':11
             }

In [8]:
##ФИЛЬТР##
DATE = interact(date_choice, DATE = sorted(hexo['ANALYTIC_DATE'].unique()))
REGION = interact(region_choice, REGION = sorted(hexo['REGION_ID'].unique()))
DISTRICT = interact(district_choice, DISTRICT = sorted(hexo['area'].unique()))
INDUSTRY = interact(industry_choice, INDUSTRY = sorted(hexo['INDUSTRY_NAME'].unique()))
BIN = interact(bin_choice, BIN = hexo['SPENDING_BIN'].unique())
CATEGORY = interact(category_choice, CATEGORY = sorted([x for x in hexo.columns.to_list() 
                                                       if ('COLOR_' in x)]))
SIZE = interact(size_choice, SIZE = sorted(hexo['size'].unique()))

interactive(children=(Dropdown(description='DATE', options=('2022-07-01',), value='2022-07-01'), Output()), _d…

interactive(children=(Dropdown(description='REGION', options=(45000000000,), value=45000000000), Output()), _d…

interactive(children=(Dropdown(description='DISTRICT', options=('Arbat', 'Otradnoe'), value='Arbat'), Output()…

interactive(children=(Dropdown(description='INDUSTRY', options=('АЗС', 'Автозапчасти', 'Автомойки', 'Аксессуар…

interactive(children=(Dropdown(description='BIN', options=('None', '2', '1', '4', '3', '5', '6', '0'), value='…

interactive(children=(Dropdown(description='CATEGORY', options=('COLOR_AVG_CHECK', 'COLOR_AVG_CHEQUE_PART', 'C…

interactive(children=(Dropdown(description='SIZE', options=('Big', 'Medium', 'Small'), value='Big'), Output())…

In [23]:
##вИЗУАЛИЗАЦИЯ##
color_size = pd.DataFrame(hexo[(hexo['area'] == district_name) & 
                         (hexo['INDUSTRY_NAME'] == industry_name) & 
                         (hexo['size'] == size_name) &
                         (hexo['SPENDING_BIN'] == bin_name) &
                         (hexo['ANALYTIC_DATE'] == date_name)])
                         
color_size = pd.DataFrame(color_size[(color_size[category_name] != str(deli)) &
                                     (color_size[category_name] != '-')][category_name])
color_size[category_name] = color_size[category_name].astype('int64').astype('str')

hexo_viz = hexo[(hexo['area'] == district_name) & 
                (hexo['INDUSTRY_NAME'] == industry_name) &
                (hexo['size'] == size_name) &
                (hexo['SPENDING_BIN'] == bin_name) &
                (hexo['ANALYTIC_DATE'] == date_name)]

hexo_viz = hexo_viz[hexo_viz.index.isin(list(color_size.index))]

color_size = color_size.reset_index(drop=True)
hexo_viz = hexo_viz.reset_index(drop=True)


def visualize_hexagons(hexagons, color_size = color_size, folium_map=None):

    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    
    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    else:
        m = folium_map
        
    k = -1    
    for polyline in polylines:
        k += 1
        my_PolyLine=folium.PolyLine(locations=polyline, 
                                    weight=dict_size.get(str(color_size.iloc[k,0])),
                                    color=dict_color.get(str(color_size.iloc[k,0])))
        m.add_child(my_PolyLine)
    return m


indic = hexo_viz[[category_name[6:], category_name]]
indic_count = indic.groupby(by = [category_name]).count().reset_index()
indic_count = indic_count.set_axis(['Bin', 'Count'], axis=1, inplace=False)
indic_min = indic.groupby(by = [category_name]).min().reset_index()
indic_max = indic.groupby(by = [category_name]).max().reset_index()
indic_all = indic_min.merge(indic_max, how = 'left', on = [category_name])
indic_all = indic_all.set_axis(['Bin', 'Min', 'Max'], axis=1, inplace=False)
indic_all['Range'] = '[' + round(indic_all['Min'],2).astype('str') + '; ' + round(indic_all['Max'], 2).astype('str') + ']'
indic_all = indic_all.merge(indic_count, how = 'left', on = ['Bin'])
indic_all['Bin'] = indic_all['Bin'].astype('int')
indic_all = indic_all[['Bin', 'Range', 'Count']].sort_values(by = ['Bin'])
print(indic_all.to_string(index=False))

visualize_hexagons(hexo_viz['CELL_ID'].to_list())

 Bin        Range  Count
   1 [0.01; 0.01]      4
   2 [0.02; 0.03]      4
   3 [0.04; 0.05]      7
   4 [0.06; 0.06]      2
   5 [0.07; 0.08]      6
   7 [0.17; 0.19]      2
  10 [0.37; 0.37]      1
